<a href="https://colab.research.google.com/github/StarMindz/Pinterest-Image-Scrapper/blob/main/Image_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pinterest Image Scrapper

This script is designed for scraping both high-quality and low-quality images from the internet using a set of search terms. It includes scripts for scraping images from Pinterest. You can specify the number of images you wish to scrape and the directory on your computer where you'd like to store them. For each search term in your list, a folder will be automatically created with the same name as the search term, and the specified number of images will be downloaded automatically.

Here, a list of popular Nigerian foods is used, making this script useful for gathering image datasets for machine learning training and general AI model training.

In [ ]:
!pip install selenium
!pip install selenium_stealth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
!import pillow

/bin/bash: line 1: import: command not found


In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import io
from datetime import datetime

In [ ]:
nigerian_dishes = [
    "Jollof Rice", "Pounded Yam", "Egusi Soup", "Meat", "Akara", "Efo Riro", "Banga Soup", "Ofada Rice and Ofada Stew",
    "Edikang Ikong Soup", "Amala", "Ogbono Soup", "Gbegiri and Ewedu Soup", "Nkwobi", "Afang Soup", "Tuwon Shinkafa", "Fried Plantain", "Miyan Taushe",
    "Oha Soup", "Masa Waina", "Beans and Plantain", "Bitterleaf Soup", "Ofe Nsala",
   "Yam Porridge", "Vegetable soup", "Ogbono Soup", "Okra soup","Pepper Soup"
    "Puff-Puff","Chin Chin", "Plantain Chips", "Meat pie", "Fish roll", "Egg Roll", "Spring Roll", "Samosa"
]

In [ ]:
pinterest_base_url = "https://www.pinterest.com/search/pins/?q="

In [ ]:
# Prepare the browser
def prepare_browser():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("start-maximized")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [ ]:
# Download images
def download_images(images, search_term, save_dir, max_images=1000):
    count = len(os.listdir(save_dir))  # Start counting from the current number of files in the directory
    for img in images:
        if count >= max_images:
            break
        src = img.get_attribute('src')
        if src and 'https' in src:  # Ensure the src attribute is valid
            try:
                img_data = requests.get(src).content
                timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
                img_file = os.path.join(save_dir, f"{search_term}_{timestamp}.jpg")
                with open(img_file, 'wb') as handler:
                    handler.write(img_data)
                count += 1
            except Exception as e:
                print(f"Could not download {src}: {e}")

In [ ]:
def download_single_image(image_url, search_term, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    if image_url and 'https' in image_url:
        try:
            img_data = requests.get(image_url).content
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
            img_file = os.path.join(save_dir, f"{search_term}_{timestamp}.jpg")
            with open(img_file, 'wb') as handler:
                handler.write(img_data)
            print(f"Image downloaded: {img_file}")
        except Exception as e:
            print(f"Could not download {image_url}: {e}")

In [ ]:
def wait_and_click(driver, by, value):
    try:
        # Wait for the element to be clickable
        element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((by, value))
        )
        # Scroll into view and click using JavaScript
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        driver.execute_script("arguments[0].click();", element)
    except:
        pass

In [ ]:
# Scrape images for a search term
def scrape_low_quality_images(driver, search_term, save_dir, max_images=1000):
    url = f'https://www.pinterest.com/search/pins/?q={search_term}'
    driver.get(url)
    print(f"Searching for {search_term} on Pinterest")

    # Scroll down to load more images
    scroll_pause_time = 2
    while True:
        images = driver.find_elements(By.TAG_NAME, 'img')
        print("Test", images)
        download_images(images, search_term, save_dir, max_images)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        if len(os.listdir(save_dir)) >= max_images:
            break


In [ ]:
def scrape_high_quality_images(driver, search_term, save_dir, max_images=1000):
    url = f'https://www.pinterest.com/search/pins/?q={search_term}'
    driver.get(url)
    print(f"Searching for {search_term} on Pinterest")
    time.sleep(10)

    downloaded_count = 0
    pins_seen = set()
    image_srcs = set()

    while downloaded_count < max_images:
        # Collect visible thumbnail XPaths
        images = [img for img in driver.find_elements(By.XPATH, '//div[@data-test-id="pin"]//img') if img.is_displayed()]

        for img in images:
            try:
                print("I got here")
                img_source = img.get_attribute('src')
                print("Current src", img_source)
                image_srcs.add(img_source)
            except:
                continue

        print("The image srcs ", image_srcs)

        # Process each image
        for img_src in image_srcs:
            if downloaded_count >= max_images:
                break

            if img_src not in pins_seen:
                print("Collected Image ", img_src)
                # Scroll to the image to make it visible
                # scroll_to_element(driver, img)
                wait_and_click(driver, By.XPATH, f'//img[@src="{img_src}"]')
                time.sleep(3)  # Wait for the pin to open

                # Re-locate the full image element
                try:
                    full_image = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//img[contains(@class, "hCL kVc L4E MIw") and not(@elementtiming="closeupImage")]'))
                    )
                except:
                    pass
                try:
                    full_image_url = full_image.get_attribute('src')
                    print("The full image src", full_image_url)
                    download_single_image(full_image_url, search_term, save_dir)
                    downloaded_count += 1
                    pins_seen.add(img_src)
                except:
                    pass

                driver.execute_script("window.history.go(-1)")
                time.sleep(2)  # Wait for the page to go back

        # Clear processed image_srcs and load more content if necessary
        image_srcs.clear()


In [ ]:
# Main function
def main():
    driver = prepare_browser()
    base_dir = r'C:\Users\STARMINDS\Desktop\Projects\Data' # Specify your base directory here
    os.makedirs(base_dir, exist_ok=True)
    for dish in nigerian_dishes:
        save_dir = os.path.join(base_dir, dish)
        os.makedirs(save_dir, exist_ok=True)
        scrape_high_quality_images(driver, dish, save_dir, max_images)
    driver.quit()

In [ ]:
max_images = 4

In [ ]:
if __name__ == '__main__':
    main()